## Data Aggregation

Load U.S. Small Business Administration (SBA) loan dataset from Kaggle and do data aggregation.
https://www.kaggle.com/mirbektoktogaraev/should-this-loan-be-approved-or-denied

In [ ]:
import pandas as pd
import numpy as np
sba_loan = pd.read_csv('..../SBAnational.csv')

### data transformations

In [ ]:
def Y_N_convert(x):
    if x == 'N':
        return 0
    elif x == 'Y':
        return 1
    return np.nan

In [ ]:
def new_exist(x):
    if x == 1:
        return 1
    elif x == 2:
        return 0
    else:
        return np.nan

In [ ]:
def year_to_int(x):
    if isinstance(x, int):
        return x
    try:
        return int(x[:4])
    except:
        return np.nan

In [ ]:
def money_to_float(x):
    try:
        return float(x.replace('$', '').replace(',', '').strip())
    except:
        return np.nan

In [ ]:
def mis_status(x):
    try:
        if 'P I F' in x:
            return 1
        elif 'CHGOFF' in x:
            return 0
    except:
        return np.nan

In [ ]:
sba_loan['ApprovalDate'] = pd.to_datetime(sba_loan['ApprovalDate'])
sba_loan['ApprovalFY'] = sba_loan['ApprovalFY'].apply(year_to_int)
sba_loan['NewExist'] = sba_loan['NewExist'].apply(new_exist).astype('Int64')
sba_loan['RevLineCr'] = sba_loan['RevLineCr'].apply(Y_N_convert).astype('Int64')
sba_loan['LowDoc'] = sba_loan['LowDoc'].map(Y_N_convert).astype('Int64')
sba_loan['ChgOffDate'] = pd.to_datetime(sba_loan['ChgOffDate'])
sba_loan['DisbursementDate'] = pd.to_datetime(sba_loan['DisbursementDate'])
sba_loan['DisbursementGross'] = sba_loan['DisbursementGross'].map(money_to_float)
sba_loan['BalanceGross'] = sba_loan['BalanceGross'].map(money_to_float)
sba_loan['MIS_Status'] = sba_loan['MIS_Status'].map(mis_status).astype('Int64')
sba_loan['ChgOffPrinGr'] = sba_loan['ChgOffPrinGr'].map(money_to_float)
sba_loan['GrAppv'] = sba_loan['GrAppv'].map(money_to_float)
sba_loan['SBA_Appv'] = sba_loan['SBA_Appv'].map(money_to_float)

In [ ]:
industry_dict = {
    11: 'Agriculture, forestry, fishing and hunting',
    21: 'Mining, quarrying, and oil and gas extraction',
    22: 'Utilities',
    23: 'Construction',
    31: 'Manufacturing',
    32: 'Manufacturing',
    33: 'Manufacturing',
    42: 'Wholesale trade',
    44: 'Retail trade',
    45: 'Retail trade',
    48: 'Transportation and warehousing',
    49: 'Transportation and warehousing',
    51: 'Information',
    52: 'Finance and insurance',
    53: 'Real estate and rental and leasing',
    54: 'Professional, scientific, and technical services',
    55: 'Management of companies and enterprises',
    56: 'Administrative and support and waste management and remediation services',
    61: 'Educational services',
    62: 'Health care and social assistance',
    71: 'Arts, entertainment, and recreation',
    72: 'Accommodation and food services',
    81: 'Other services (except public administration)',
    92: 'Public administration'
}

def industry(x):
    try:
        return industry_dict[int(str(x)[:2])]
    except:
        return np.nan

sba_loan['Industry'] = sba_loan['NAICS'].map(industry)

In [ ]:
sba_loan.dtypes

1. compute the median gross amount of loan approved by the bank GrAppv

In [ ]:
sba_loan.groupby('Industry')['GrAppv'].median()

2. compute the proportion of loans which have status as charged off (CHGOFF)

In [ ]:
1-sba_loan.groupby('Industry')['MIS_Status'].mean()

3. Divide the data of ApprovalFY into bins: [1960, 1970), [1970, 1980), [1980, 1990), [1990, 2000), [2000, 2010), [2010, 2020) to different groups and compute the proportion/percentage of loans that have been declared to be in default (CHGOFF in MIS_Status)

In [ ]:
bins = [1960, 1970, 1980, 1990, 2000, 2010]
sba_loan['ApprovalFY'] = pd.cut(sba_loan['ApprovalFY'], bins, right=False)

In [ ]:
1-sba_loan.groupby(['ApprovalFY','LowDoc'])['MIS_Status'].mean()

In [ ]:
1-sba_loan.groupby(['ApprovalFY','LowDoc'])['MIS_Status'].mean()[:,1]

4. Divide the data of ApprovalFY into bins: [1960, 1970), [1970, 1980), [1980, 1990), [1990, 2000), [2000, 2010), [2010, 2020) and compute:

  -The number of observations in that group.

  -The median gross amount of loan approved by the bank GrAppv.

  -The median loan term in months Term.

  -The percentage of loans that were fully guaranteed by the SBA (that is GrAppv = SBA_Appv).

In [ ]:
def fully_guaranteed(df):
    if df[0]-df[1] == 0:
        return 1
    else:
        return 0

In [ ]:
sba_loan['fully_guaranteed'] = sba_loan[['SBA_Appv','GrAppv']].apply(fully_guaranteed, axis=1)

In [ ]:
grouped = sba_loan.groupby(['ApprovalFY','UrbanRural'])

In [ ]:
grouped.agg({
    'Name': 'count',
    'GrAppv': 'median',
    'Term': 'median',
    'fully_guaranteed':'mean'
})